In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/*'

In [6]:
file_list = []
out_file = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and '0053m1' in p:
        file_list.append(p)
        frame = p.split('_')[-3][:5]
        out_file.append('0053m1_r'+ frame + '_lens_szap.fits')

In [13]:
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[355, 405])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d()     #outfile=out_file[i])
    filled_data, mask = szap(spec.data)
    pf.PrimaryHDU(filled_data).writeto(out_file[i])


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits (HDU=12)
The input dataset was trimmed
 xrange: 355:406.  yrange: 0:4096
Final data dimensions (x y): 51 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 51


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



sky model will be generated from data

 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From this point sky subtracted data will be used

Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026b-0053m1_LRISr_2020Dec13T045626.045.fits
-------------------------------

In [12]:
def szap(sp_1):

    sp_2 = pf.open('0053m1_lens_median_image.fits')[0].data
    mask = np.zeros(sp_1.shape, dtype=bool)
    #print(sp_1.shape)

    for k in range(0, sp_1.shape[0], 400):
        for i in range(0, sp_1.shape[1], 5):
            sp1 = sp_1[k:k+400, i:i+5]
            sp2 = sp_2[k:k+400, i:i+5]
            count1, bins1 = np.histogram(sp1.flatten(), bins=1000)
            count2, bins2 = np.histogram(sp2.flatten(), bins=1000)
            sm1 = 0
            sm2 = 0
            cnt1 = 0
            cnt2 = 0
            tot_pix =sp1.flatten().shape[0]

            for j,p in enumerate(count1):
                if p > cnt1:
                    cnt1 = p
                    max_bin1 = bins1[j+1]
                sm1 += p
                if sm1/tot_pix >= 0.8 :
                    border1 = 0.5*(bins1[j+1] + bins1[j])
                    break
            #print(border)
            for j,p in enumerate(count2):
                if p > cnt2:
                    cnt2 = p
                    max_bin2 = 0.5*(bins2[j+1] + bins2[j])
                sm2 += p
                if sm2/tot_pix >= 0.8 :
                    border2 = bins2[j+1]
                    break

            a = np.where(sp1>border1)
            a =np.transpose(a)

            if abs(border1 - max_bin1) < 50 :
                mp = 50    #/ abs(border1 - max_bin1)
            elif abs(border1 - max_bin1) < 100 :
                mp = 150
            elif abs(border1 - max_bin1) < 300:
                mp = 200
            else:
                mp = 400

            for j, p in enumerate(a):

                #print(sp2[p[0]][p[1]])
                if sp2[p[0]][p[1]] >0 :
                    #pass 
                    if sp1[p[0]][p[1]] > (sp2[p[0]][p[1]]+mp) :
                        mask[k+p[0]][i+p[1]] =1 
                else:
                    if sp1[p[0]][p[1]] > 3*border1 :
                        mask[k+p[0]][i+p[1]] =1
                        
    #print(mask)
    sp_1[mask] = sp_2[mask]
    
    mf =  (sp_1 > 200) | (sp_1 < -100)
    
    for k, p in enumerate(np.transpose(np.where(mf==True))):
        mask[p[0]][p[1]] = 1
        reg = sp_1[p[0]-10: p[0]+10, p[1]:p[1]+1]
        sp_1[p[0]][p[1]] = np.median(reg)
                
        #d[mask] = fill[mask] 
        #mm = m.astype('bool') | mask
    
    return sp_1, mask.astype(int)

In [5]:
## median image
spec_list = []
for i, p in enumerate(file_list):
    spec = spec2d.Spec2d(p, hext=12, xtrim=[355, 405])
    spec.set_dispaxis('y')
    spec.subtract_sky_2d()
    spec_list.append(spec.data)
median_im = np.median(np.asarray(spec_list), axis=0)
pf.PrimaryHDU(median_im).writeto('0053m1_lens_median_image.fits')


Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026a-0053m1_LRISr_2020Dec13T045626.045.fits (HDU=12)
The input dataset was trimmed
 xrange: 355:406.  yrange: 0:4096
Final data dimensions (x y): 51 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 51


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



sky model will be generated from data

 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From this point sky subtracted data will be used

Loading file ../galaxy_spectra/LRIS_red_data_201213/keck_lris_red_A/Science/spec2d_r201213_0026b-0053m1_LRISr_2020Dec13T045626.045.fits
-------------------------------

In [ ]:
spec.szap() #outfile='0053m1_lens_szap.fits'

In [ ]:
spec.do_waverect(doplot=True)

In [ ]:
profile = spec.spatial_profile(pixrange=[1600, 3500])

In [ ]:
spec.find_and_trace()

In [ ]:
spec.display_spec()